# Koosta regressioonimudel: Alusta R-i ja Tidymodelsiga regressioonimudelite jaoks


## Sissejuhatus regressiooni - Õppetund 1

#### Paneme selle perspektiivi

✅ Regressioonimeetodeid on mitut tüüpi ja millise valite, sõltub vastusest, mida otsite. Kui soovite ennustada tõenäolist pikkust teatud vanuses inimese jaoks, kasutaksite `lineaarset regressiooni`, kuna otsite **arvväärtust**. Kui olete huvitatud avastamast, kas teatud köögitüüp peaks olema vegan või mitte, otsite **kategooria määramist**, seega kasutaksite `logistilist regressiooni`. Logistilisest regressioonist õpite hiljem rohkem. Mõelge veidi küsimustele, mida saate andmetelt küsida, ja milline neist meetoditest oleks sobivam.

Selles osas töötate [väikese diabeedi andmestikuga](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Kujutage ette, et soovite testida ravi diabeediga patsientide jaoks. Masinõppe mudelid võivad aidata teil kindlaks teha, millised patsiendid reageeriksid ravile paremini, tuginedes muutujate kombinatsioonidele. Isegi väga lihtne regressioonimudel, kui seda visualiseerida, võib näidata teavet muutujate kohta, mis aitaksid teil korraldada teoreetilisi kliinilisi katseid.

Sellega öeldes, alustame selle ülesandega!

<p >
   <img src="../../../../../../translated_images/et/encouRage.e75d5fe0367fb913.jpg"
   width="630"/>
   <figcaption>Kunstiteos: @allison_horst</figcaption>


## 1. Tööriistakomplekti laadimine

Selle ülesande jaoks vajame järgmisi pakette:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) on [R-pakettide kogum](https://www.tidyverse.org/packages), mis on loodud andmeteaduse kiiremaks, lihtsamaks ja lõbusamaks muutmiseks!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) raamistik on [pakettide kogum](https://www.tidymodels.org/packages/), mis on mõeldud modelleerimiseks ja masinõppeks.

Saate need paigaldada järgmiselt:

`install.packages(c("tidyverse", "tidymodels"))`

Allolev skript kontrollib, kas teil on selle mooduli täitmiseks vajalikud paketid olemas, ja paigaldab need, kui mõni neist puudub.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Nüüd laadime need suurepärased paketid ja teeme need meie praeguses R-i sessioonis kättesaadavaks. (See on lihtsalt illustratsiooniks, `pacman::p_load()` tegi seda juba teie eest)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Diabeedi andmestik

Selles harjutuses rakendame oma regressioonioskusi, tehes ennustusi diabeedi andmestiku põhjal. [Diabeedi andmestik](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) sisaldab `442 näidist` diabeediga seotud andmeid, kus on 10 ennustavat tunnust, nagu `vanus`, `sugu`, `kehamassiindeks`, `keskmine vererõhk` ja `kuus vere seerumi mõõtmist`, ning tulemuse muutuja `y`: kvantitatiivne näitaja haiguse progresseerumisest üks aasta pärast algtaset.

|Vaatluste arv|442|
|----------------------|:---|
|Ennustavate tunnuste arv|Esimesed 10 veergu on numbrilised ennustavad tunnused|
|Tulemus/Sihtmärk|11. veerg on kvantitatiivne näitaja haiguse progresseerumisest üks aasta pärast algtaset|
|Ennustavate tunnuste info|- vanus aastates
||- sugu
||- bmi kehamassiindeks
||- bp keskmine vererõhk
||- s1 tc, kogu seerumi kolesterool
||- s2 ldl, madala tihedusega lipoproteiinid
||- s3 hdl, kõrge tihedusega lipoproteiinid
||- s4 tch, kogu kolesterool / HDL
||- s5 ltg, tõenäoliselt seerumi triglütseriidide taseme logaritm
||- s6 glu, veresuhkru tase|



> 🎓 Pea meeles, et see on juhendatud õpe ja meil on vaja nimetatud sihtmärki 'y'.

Enne kui saad R-is andmeid manipuleerida, pead andmed importima R-i mällu või looma ühenduse andmetega, mida R saab kasutada andmete kaugjuurdepääsuks.

> [readr](https://readr.tidyverse.org/) pakett, mis on osa Tidyverse'ist, pakub kiiret ja kasutajasõbralikku viisi ristkülikukujuliste andmete lugemiseks R-i.

Nüüd laadime diabeedi andmestiku, mis on saadaval sellel URL-il: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Samuti teeme andmetele kontrolli `glimpse()` abil ja kuvame esimesed 5 rida `slice()` abil.

Enne kui edasi liigume, tutvustame midagi, mida kohtad R-i koodis sageli 🥁🥁: toruoperaatorit `%>%`

Toruoperaator (`%>%`) teostab toiminguid loogilises järjestuses, edastades objekti edasi funktsiooni või väljendisse. Toruoperaatorit võib mõelda kui "ja siis" sinu koodis.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` näitab meile, et andmetes on 442 rida ja 11 veergu, kus kõik veerud on andmetüüpi `double`.

<br>

> glimpse() ja slice() on funktsioonid [`dplyr`](https://dplyr.tidyverse.org/) paketis. Dplyr, mis on osa Tidyverse'ist, on andmete manipuleerimise grammatika, mis pakub ühtset komplekti tegusõnu, mis aitavad lahendada kõige tavalisemaid andmete manipuleerimise väljakutseid.

<br>

Nüüd, kui meil on andmed olemas, keskendume ühele tunnusele (`bmi`), mida sihtida selles harjutuses. Selleks peame valima soovitud veerud. Kuidas seda teha?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) võimaldab meil *valida* (ja soovi korral ümber nimetada) veerge andmeraamis.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Treening- ja testimisandmed

Juhendatud õppimises on tavaks *jagada* andmed kaheks alamhulgaks: (tavaliselt suurem) komplekt, millega mudelit treenitakse, ja väiksem "tagasihoitud" komplekt, millega hinnatakse, kuidas mudel toimis.

Nüüd, kui andmed on valmis, saame vaadata, kas masin suudab aidata määrata loogilise jaotuse selle andmestiku numbrite vahel. Selleks saame kasutada [rsample](https://tidymodels.github.io/rsample/) paketti, mis on osa Tidymodels raamistikust, et luua objekt, mis sisaldab teavet selle kohta, *kuidas* andmed jagada, ning seejärel kasutada kahte rsample funktsiooni, et eraldada loodud treening- ja testimiskomplektid:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Treeni lineaarse regressiooni mudel Tidymodelsiga

Nüüd oleme valmis oma mudelit treenima!

Tidymodelsis määratakse mudelid `parsnip()` abil, määratledes kolm kontseptsiooni:

-   Mudeli **tüüp** eristab mudeleid, nagu lineaarne regressioon, logistiline regressioon, otsustuspuu mudelid ja nii edasi.

-   Mudeli **režiim** sisaldab tavalisi valikuid, nagu regressioon ja klassifikatsioon; mõned mudelitüübid toetavad mõlemat, samas kui mõned ainult ühte režiimi.

-   Mudeli **mootor** on arvutuslik tööriist, mida kasutatakse mudeli sobitamiseks. Sageli on need R-i paketid, nagu **`"lm"`** või **`"ranger"`**

See modelleerimisteave salvestatakse mudeli spetsifikatsioonis, nii et loome selle!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Pärast seda, kui mudel on *määratletud*, saab mudelit `hinnata` või `treenida` funktsiooni [`fit()`](https://parsnip.tidymodels.org/reference/fit.html) abil, tavaliselt kasutades valemit ja mõningaid andmeid.

`y ~ .` tähendab, et me sobitame `y` ennustatava suuruse/eesmärgina, mida selgitavad kõik ennustajad/tunnused ehk `.` (antud juhul on meil ainult üks ennustaja: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Mudeli väljundist näeme treeningu käigus õpitud koefitsiente. Need esindavad parima sobivusega joone koefitsiente, mis annavad meile tegeliku ja prognoositud muutuja vahelise väikseima üldise vea.
<br>

## 5. Ennustuste tegemine testandmestikul

Nüüd, kui oleme mudeli treeninud, saame seda kasutada haiguse progresseerumise y ennustamiseks testandmestiku põhjal, kasutades [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Seda kasutatakse andmegruppide vahelise joone joonistamiseks.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Jee! 💃🕺 Me just treenisime mudelit ja kasutasime seda ennustuste tegemiseks!

Ennustuste tegemisel on tidymodels'i tava alati luua tulemuste tibble/andmeraamistik standardiseeritud veerunimedega. See muudab algandmete ja ennustuste kombineerimise lihtsaks kasutatavasse formaati edasisteks toiminguteks, nagu näiteks graafikute loomine.

`dplyr::bind_cols()` ühendab tõhusalt mitu andmeraamistikku veergude kaupa.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Modelleerimistulemuste visualiseerimine

Nüüd on aeg seda visuaalselt näha 📈. Loome hajuvusdiagrammi kõigist testkomplekti `y` ja `bmi` väärtustest ning kasutame ennustusi, et joonistada joon kõige sobivamasse kohta mudeli andmegruppide vahel.

R-il on mitu süsteemi graafikute loomiseks, kuid `ggplot2` on üks elegantsemaid ja mitmekülgsemaid. See võimaldab graafikuid koostada, **ühendades sõltumatuid komponente**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

> ✅ Mõtle veidi, mis siin toimub. Sirgjoon läbib paljusid väikeseid andmepunkte, aga mida see täpselt teeb? Kas sa näed, kuidas saaksid seda joont kasutada, et ennustada, kuhu uus, seni nägemata andmepunkt peaks graafiku y-telje suhtes sobituma? Proovi sõnastada selle mudeli praktiline kasutus.

Palju õnne, sa ehitasid oma esimese lineaarse regressioonimudeli, tegid sellega ennustuse ja kuvad selle graafikul!



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
